In [6]:
import requests
import pandas as pd

In [7]:
base_url = 'https://www.amfiindia.com/modules/'

In [8]:
mf_list = pd.read_json('data/mf_list.json')
mf_list.set_index('id', inplace=True)
mf_list.sort_index(inplace=True)
mf_list

,name
id,
1,Alliance Capital Mutual Fund
2,Standard Chartered Mutual Fund
3,Aditya Birla Sun Life Mutual Fund
4,Baroda BNP Paribas Mutual Fund
6,DSP Mutual Fund
...,...
76,Helios Mutual Fund
77,Zerodha Mutual Fund
78,Old Bridge Mutual Fund


In [9]:
# search in df[col_name] by splitting the search string and sorting the results by the number of matches of n-grams
def search(df, col_name, search_string):
    dfcopy = df.copy()
    words = search_string.lower().split()
    bigrams = [' '.join(words[i:i+2]) for i in range(len(words)-1)]
    trigrams = [' '.join(words[i:i+3]) for i in range(len(words)-2)]

    def score(row):
        name = row[col_name].lower()
        score = 0
        for word in words:
            if word in name:
                score += 1
        for bigram in bigrams:
            if bigram in name:
                score += 2
        for trigram in trigrams:
            if trigram in name:
                score += 3
        return score
    
    dfcopy['score'] = dfcopy.apply(score, axis=1)
    return dfcopy[dfcopy['score'] > 0].sort_values('score', ascending=False).drop('score', axis=1)


def search_mf(search_string):
    return search(mf_list, 'name', search_string)

search_mf('sbi')

,name
id,
22,SBI Mutual Fund


In [10]:
schemes_response = requests.post(base_url + 'NavHistorySchemeNav', data={'ID': '22'}, headers={"X-Requested-With": "XMLHttpRequest"})
schemes_json = schemes_response.json()
schemes_df = pd.DataFrame(schemes_json)
schemes_df.drop(['Selected'], axis=1, inplace=True)
schemes_df.set_index('Value', inplace=True)
schemes_df.sort_index(inplace=True)
schemes_df

,Text
Value,
100638,SBI Magnum Income Fund - Regular Plan - Half Y...
100639,SBI Magnum Income Fund-REGULAR PLAN-Growth
100640,SBI Magnum Income Fund-REGULAR PLAN-Bonus
100641,SBI MAGNUM ULTRA SHORT DURATION FUND - REGULAR...
100642,Z-OLD-SBI MICF DIVIDEND PLAN-Dividend-Holdings...
...,...
153253,SBI Nifty Bank Index Fund-Regular Plan- Growth
153320,SBI Nifty IT Index Fund - Direct Plan - Growth
153321,SBI Nifty IT Index Fund - Direct Plan - Income...


In [15]:
def search_schemes(search_string):
    return search(schemes_df, 'Text', search_string)

search_schemes('equity hybrid direct')

,Text
Value,
119604,SBI Equity Hybrid Fund - Direct Plan - Income ...
119609,SBI EQUITY HYBRID FUND - DIRECT PLAN - Growth
101551,SBI Equity Hybrid Fund - Regular Plan - Income...
102885,SBI EQUITY HYBRID FUND - REGULAR PLAN -Growth
131338,SBI EQUITY OPPORTUNITIES FUND - SERIES I - DIR...
...,...
153154,SBI Quant Fund- Direct Plan- Growth
153250,SBI Nifty Bank Index Fund-Direct Plan- Growth
153252,SBI Nifty Bank Index Fund-Direct Plan- Income ...


In [12]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
htmldata_response = requests.post(
    base_url + 'NavHistoryPeriod',
    data={
        'mfID': '22',
        'scID': '119609',
        'fDate': '01-Mar-2020',
        'tDate': '23-Feb-2025'
    },
    headers={"X-Requested-With": "XMLHttpRequest"}
)
try:
    df = pd.read_html(htmldata_response.text)
    df = df[0]
    df.index = df.iloc[:, -1].values
    df = df.iloc[:, :1]
    df.index.name = 'date'
    df.columns = ['nav']
    df.index = pd.to_datetime(df.index, format="%d-%b-%Y")
    display(df)
except:
    print(htmldata_response.text)
    print('Error in fetching data')

C:\Users\vatsa\AppData\Local\Temp\ipykernel_293924\3563056005.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(htmldata_response.text)


,nav
date,
2020-03-02,152.6254
2020-03-03,154.1598
2020-03-04,153.1530
2020-03-05,153.8289
2020-03-06,151.9624
...,...
2025-02-17,297.9564
2025-02-18,297.7959
2025-02-19,299.0559
